<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/annotation_import/tiled.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/annotation_import/tiled.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Tiled Imagery Annotation Import
* This notebook will provide examples of each supported annotation type for tiled imagery assets. It will cover the following:
    * Model-assisted labeling - used to provide pre-annotated data for your labelers. This will enable a reduction in the total amount of time to properly label your assets. Model-assisted labeling does not submit the labels automatically, and will need to be reviewed by a labeler for submission.
    * Label Import - used to provide ground truth labels. These can in turn be used and compared against prediction labels, or used as benchmarks to see how your labelers are doing.

* For information on what types of annotations are supported per data type, refer to this documentation:
    * https://docs.labelbox.com/docs/model-assisted-labeling#option-1-import-via-python-annotation-types-recommended

* Notes:
    * The example in the notebook uses a heuristic to prelabel water in the imagery
    * If you are importing more than 1,000 mask annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
    * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.

In [ ]:
!pip install -q 'labelbox[data]'

In [ ]:
import os

import uuid
import numpy as np
from PIL import Image
import cv2
import ndjson

from labelbox.data.annotation_types.data.tiled_image import TiledBounds, TiledImageData, TileLayer, EPSG, EPSGTransformer
from labelbox.data.serialization.ndjson.converter import NDJsonConverter
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, MaskData,
    Rectangle, Point, Line, Mask, Polygon,
    Radio, Checklist, Text,
    ClassificationAnnotation, ClassificationAnswer
)
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client, LabelingFrontend, LabelImport, MALPredictionImport

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

INFO:labelbox.client:Initializing Labelbox client at 'https://api.labelbox.com/graphql'


---- 
### Steps
1. Make sure project is setup
2. Collect annotations
3. Upload

### Project Setup

We will be creating two projects, one for model-assisted labeling, and one for label imports

In [ ]:
ontology_builder = OntologyBuilder(
    tools=[
        Tool(tool=Tool.Type.BBOX, name="box"),
        Tool(tool=Tool.Type.LINE, name="line"),
        Tool(tool=Tool.Type.POINT, name="point"),
        Tool(tool=Tool.Type.POLYGON, name="polygon"),    
        Tool(tool=Tool.Type.SEGMENTATION, name="mask")],
    classifications=[
        Classification(class_type=Classification.Type.TEXT, instructions="text"),
        Classification(class_type=Classification.Type.CHECKLIST, instructions="checklist", options=[
            Option(value="first_checklist_answer"),
            Option(value="second_checklist_answer")            
        ]),
        Classification(class_type=Classification.Type.RADIO, instructions="radio", options=[
            Option(value="first_radio_answer"),
            Option(value="second_radio_answer")
        ]),                                      
])


In [ ]:
# This notebook only uploads data for a single dataRow.

# Select the region to label within the image
top_left_bound = Point(x=-122.31764674186705, y=37.87276155898985)
bottom_right_bound = Point(x=-122.31635199317932, y=37.87398109727749)

epsg = EPSG.EPSG4326
bounds = TiledBounds(epsg=epsg, bounds=[top_left_bound, bottom_right_bound])
tile_layer = TileLayer(
    url=
    "https://public-tiles.dronedeploy.com/1499994155_DANIELOPENPIPELINE_ortho_qfs/{z}/{x}/{y}.png?Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9wdWJsaWMtdGlsZXMuZHJvbmVkZXBsb3kuY29tLzE0OTk5OTQxNTVfREFOSUVMT1BFTlBJUEVMSU5FX29ydGhvX3Fmcy8qIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUaW1lIjoyMTQ1OTE0MTE4fX19XX0_&Signature=O~50rrGXdEC6Hi8jPJ3dbT~UtBd7Cw6iQPTxdJ8LU2IaoxeP22R3JpKPkLN3T3~Lcw3CyX7uft2Baj0MH93qUoCYyN~~jNX3OMkYV2jbrHDezf6zQRHAabXX-L2bL-JEGfFL6z3DWccOFeCH56CuhgC29k5CJx7I34P-LQJdnAUsA-KaqKH1IyYsHStRIfmMzdXNAWU58FTfqVljq9SbKXxfgdr2SZ~7VgLaZ8IhA0WnlKUo-JgqTd~jYa5mGCpR8351IMK0aMuY4Mld4SOXssQ-rOtlZtypvo8FDp474TlGIEGz5PHxGOPsqLPF19hEYTgoPqsUj8QEuiTfg-cmsg__&Key-Pair-Id=APKAJXGC45PGQXCMCXSA"
)

tiled_image_data = TiledImageData(tile_layer=tile_layer,
                                  tile_bounds=bounds,
                                  zoom_levels=[17, 23])

In [ ]:
# Lets setup the project
# Note: see Ontology, Project, and Project_setup notebooks for more information on this section.

mal_project = client.create_project(name="tms_mal_project")
li_project = client.create_project(name="tms_label_import_project")
dataset = client.create_dataset(name="tms_annotation_import_dataset")

# DataRows can only be created as bulk operations for tiled imagery data
dataset.create_data_rows([tiled_image_data.asdict()])
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))

mal_project.setup(editor, ontology_builder.asdict())
mal_project.datasets.connect(dataset)

li_project.setup(editor, ontology_builder.asdict())
li_project.datasets.connect(dataset)

### Create Label using Annotation Type Objects
* It is recommended to use the Python SDK's annotation types for importing into Labelbox.

### Object Annotations

In [ ]:
#we will create every object other than mask and polygon, which will be the next section
point = Point(x=100,y=100)
point_annotation = ObjectAnnotation(value=point, name="point")

rectangle = Rectangle(start=Point(x=30,y=30), end=Point(x=200,y=200))
rectangle_annotation = ObjectAnnotation(value=rectangle, name="box")

line = Line(points=[Point(x=60,y=70), Point(x=65,y=100), Point(x=80,y=130), Point(x=40,y=200)])
line_annotation = ObjectAnnotation(value=line, name="line")

In [ ]:
hsv = cv2.cvtColor(tiled_image_data.value, cv2.COLOR_RGB2HSV)
mask = cv2.inRange(hsv, (50, 10, 25), (100, 150, 255))
kernel = np.ones((5, 5), np.uint8)
mask = cv2.erode(mask, kernel)
mask = cv2.dilate(mask, kernel)

mask_annotation = MaskData.from_2D_arr(mask)
mask_data = Mask(mask=mask_annotation, color=[255, 255, 255])
mask_annotation = ObjectAnnotation(value=mask_data, name="mask")


#### Converting from pixel coordinates to WGS84 coordinates
* We can convert our mask annotation to a different coordinate space.
* In this example, our Polygon points will reflect WGS84
* This can be repeated for the other Annotation Types if desired.

In [ ]:
#generate the pixel bounds for our transformer
h, w, _ = tiled_image_data.value.shape
pixel_bounds = TiledBounds(epsg=EPSG.SIMPLEPIXEL,
                           bounds=[Point(x=0, y=0),
                                   Point(x=w, y=h)])

transformer = EPSGTransformer.create_pixel_to_geo_transformer(
    src_epsg=pixel_bounds.epsg,
    pixel_bounds=pixel_bounds,
    geo_bounds=tiled_image_data.tile_bounds,
    zoom=17)

#transform from pixel space back to EPSG 4326
pixel_polygons = mask_data.shapely.simplify(3)
geo_polygon = transformer(Polygon.from_shapely(pixel_polygons))
polygon_annotation = ObjectAnnotation(value=geo_polygon, name="polygon")

### Classification Annotations

In [ ]:
text = Text(answer="the answer to the text question")
text_annotation = ClassificationAnnotation(value=text, name="text")

checklist = Checklist(answer=[ClassificationAnswer(name="first_checklist_answer"),ClassificationAnswer(name="second_checklist_answer")])
checklist_annotation = ClassificationAnnotation(value=checklist, name="checklist")

radio = Radio(answer = ClassificationAnswer(name = "second_radio_answer"))
radio_annotation = ClassificationAnnotation(value=radio, name="radio")

### Create a Label object with all of our annotations

In [ ]:
datarow_id = next(dataset.data_rows()).uid
tiled_image_data.uid = datarow_id

label = Label(
    data=tiled_image_data,
    annotations = [
        point_annotation, rectangle_annotation, line_annotation, polygon_annotation, mask_annotation,        
        text_annotation, checklist_annotation, radio_annotation
    ]
)

# Create urls to mask data for upload
def signing_function(obj_bytes: bytes) -> str:
    url = client.upload_data(content=obj_bytes, sign=True)
    return url


label.add_url_to_masks(signing_function)

label.__dict__

{'uid': None,
 'data': TextData(file_path=None,text=None,url=None),
 'annotations': [ObjectAnnotation(name='point', feature_schema_id=None, extra={}, value=Point(extra={}, x=100.0, y=100.0), classifications=[]),
  ObjectAnnotation(name='box', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=30.0, y=30.0), end=Point(extra={}, x=200.0, y=200.0)), classifications=[]),
  ObjectAnnotation(name='line', feature_schema_id=None, extra={}, value=Line(extra={}, points=[Point(extra={}, x=60.0, y=70.0), Point(extra={}, x=65.0, y=100.0), Point(extra={}, x=80.0, y=130.0), Point(extra={}, x=40.0, y=200.0)]), classifications=[]),
  ObjectAnnotation(name='polygon', feature_schema_id=None, extra={}, value=Polygon(extra={}, points=[Point(extra={}, x=-122.31708419322968, y=37.87397262833122), Point(extra={}, x=-122.31677046418189, y=37.87357458675811), Point(extra={}, x=-122.31661900877953, y=37.87345602076741), Point(extra={}, x=-122.31652164459229, y=37.87327817142355),

### Model Assisted Labeling 

To do model-assisted labeling, we need to convert a Label object into an NDJSON. 

This is easily done with using the NDJSONConverter class

We will create a Label called mal_label which has the same original structure as the label above

Notes:
* Each label requires a valid feature schema id. We will assign it using our built in `assign_feature_schema_ids` method
* the NDJsonConverter takes in a list of labels

In [ ]:
mal_label = Label(
    data=tiled_image_data,
    annotations = [
        point_annotation, rectangle_annotation, line_annotation, polygon_annotation, mask_annotation,
        text_annotation, checklist_annotation, radio_annotation
    ]
)


label.add_url_to_masks(signing_function)

mal_label.assign_feature_schema_ids(ontology_builder.from_project(mal_project))

ndjson_labels = list(NDJsonConverter.serialize([mal_label]))

ndjson_labels

[{'uuid': 'acb45454-429d-4654-aa19-a822a2ed0de4',
  'dataRow': {'id': 'ckzyd7bqd00420zq52ms8aaiw'},
  'schemaId': 'ckzyd7cyv190k0zct729ufwb4',
  'classifications': [],
  'point': {'x': 100.0, 'y': 100.0}},
 {'uuid': '7dead30c-6f6d-4b10-867c-00dc3afe9a88',
  'dataRow': {'id': 'ckzyd7bqd00420zq52ms8aaiw'},
  'schemaId': 'ckzyd7cyu190g0zcth1dl5xjt',
  'classifications': [],
  'bbox': {'top': 30.0, 'left': 30.0, 'height': 170.0, 'width': 170.0}},
 {'uuid': '752e8a96-fcfd-49de-9476-4a6e98050b14',
  'dataRow': {'id': 'ckzyd7bqd00420zq52ms8aaiw'},
  'schemaId': 'ckzyd7cyu190i0zctber1cmun',
  'classifications': [],
  'line': [{'x': 60.0, 'y': 70.0},
   {'x': 65.0, 'y': 100.0},
   {'x': 80.0, 'y': 130.0},
   {'x': 40.0, 'y': 200.0}]},
 {'uuid': 'bcdd8a0b-5627-42df-bfea-217e98895f8f',
  'dataRow': {'id': 'ckzyd7bqd00420zq52ms8aaiw'},
  'schemaId': 'ckzyd7cyv190m0zctd1zvf120',
  'classifications': [],
  'polygon': [{'x': -122.31708419322968, 'y': 37.87397262833122},
   {'x': -122.31677046418189, 

In [ ]:
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = mal_project.uid, 
    name="upload_label_import_job", 
    predictions=ndjson_labels)

In [ ]:
print("Errors:", upload_job.errors)

INFO:labelbox.schema.annotation_import:Sleeping for 10 seconds...


Errors: []


### Label Import

Label import is very similar to model-assisted labeling. We will need to re-assign the feature schema before continuing, 
but we can continue to use our NDJSonConverter

We will create a Label called li_label which has the same original structure as the label above

In [ ]:
#for the purpose of this notebook, we will need to reset the schema ids of our checklist and radio answers
checklist = Checklist(answer=[ClassificationAnswer(name="first_checklist_answer"),ClassificationAnswer(name="second_checklist_answer")])
checklist_annotation = ClassificationAnnotation(value=checklist, name="checklist")
radio = Radio(answer = ClassificationAnswer(name = "second_radio_answer"))
radio_annotation = ClassificationAnnotation(value=radio, name="radio")

li_label = Label(
    data=tiled_image_data,
    annotations = [
        point_annotation, rectangle_annotation, line_annotation, polygon_annotation, mask_annotation,
        text_annotation, checklist_annotation, radio_annotation
    ]
)

li_label.assign_feature_schema_ids(ontology_builder.from_project(li_project))

ndjson_labels = list(NDJsonConverter.serialize([li_label]))

ndjson_labels

[{'uuid': 'eb240044-f6a5-46ed-86dc-814492b3c69e',
  'dataRow': {'id': 'ckzyd7bqd00420zq52ms8aaiw'},
  'schemaId': 'ckzyd7e1419bp0zbn1wlnh3k8',
  'classifications': [],
  'point': {'x': 100.0, 'y': 100.0}},
 {'uuid': '5a19a787-b3ee-4b5d-bb1a-331ae0f81fe6',
  'dataRow': {'id': 'ckzyd7bqd00420zq52ms8aaiw'},
  'schemaId': 'ckzyd7e1419bl0zbna6m1afsg',
  'classifications': [],
  'bbox': {'top': 30.0, 'left': 30.0, 'height': 170.0, 'width': 170.0}},
 {'uuid': 'c5607c9b-2e16-47fd-bbd2-70d6644955a7',
  'dataRow': {'id': 'ckzyd7bqd00420zq52ms8aaiw'},
  'schemaId': 'ckzyd7e1419bn0zbn7u7j9c0d',
  'classifications': [],
  'line': [{'x': 60.0, 'y': 70.0},
   {'x': 65.0, 'y': 100.0},
   {'x': 80.0, 'y': 130.0},
   {'x': 40.0, 'y': 200.0}]},
 {'uuid': 'ea67276c-10fd-4605-833b-a3f87d609984',
  'dataRow': {'id': 'ckzyd7bqd00420zq52ms8aaiw'},
  'schemaId': 'ckzyd7e1419br0zbn5uo5190v',
  'classifications': [],
  'polygon': [{'x': -122.31708419322968, 'y': 37.87397262833122},
   {'x': -122.31677046418189, 

In [ ]:
upload_job = LabelImport.create_from_objects(
    client = client, 
    project_id = li_project.uid, 
    name="upload_label_import_job", 
    labels=ndjson_labels)

In [ ]:
print("Errors:", upload_job.errors)

INFO:labelbox.schema.annotation_import:Sleeping for 10 seconds...


Errors: []
